## Análisis Predictivo: Reglas de Asociación análisis de la cesta de ventas

Las reglas de asociación son un tipo de análisis de minería de datos que se utiliza para encontrar patrones o relaciones entre variables en grandes conjuntos de datos. El objetivo de las reglas de asociación es descubrir las relaciones entre variables que ocurren con mayor frecuencia en un conjunto de datos. Esto se logra mediante la identificación de patrones de co-ocurrencia de los elementos de un conjunto de datos.

El algoritmo de apriori es uno de los métodos más utilizados para extraer reglas de asociación de un conjunto de datos. Este algoritmo se utiliza comúnmente en análisis de mercado y recomendaciones de productos, y es muy útil para descubrir patrones en grandes conjuntos de datos.

El algoritmo de apriori utiliza la frecuencia de los patrones para extraer las reglas de asociación. Primero, encuentra todos los conjuntos de elementos que ocurren con una frecuencia mayor o igual a un umbral predefinido (el soporte mínimo). Luego, utiliza estos conjuntos para construir reglas de asociación. Cada regla de asociación consta de un antecedente y un consecuente. El antecedente es un conjunto de elementos que ocurren juntos con frecuencia, mientras que el consecuente es otro conjunto de elementos que también ocurre con frecuencia y que se relaciona con el antecedente.

En resumen, el algoritmo de apriori utiliza la frecuencia de los patrones para extraer las reglas de asociación. Estas reglas son útiles para descubrir relaciones entre variables y para identificar patrones de comportamiento en grandes conjuntos de datos.

### Carga de las librerías python necesarias

In [35]:
import pyodbc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mlxtend.frequent_patterns import apriori, association_rules

plt.rcParams['figure.figsize'] = [15, 8]

import warnings
warnings.filterwarnings('ignore')

### Extracción de los datos desde SQL Server

In [36]:
conn_str = pyodbc.connect('Driver={SQL Server};' 'Server=LENOVO\SQLEXPRESS;''Database=AdventureWorks2019;''Trusted_Connection=yes;')
input_query = '''SELECT 
     T0.SalesOrderID AS Factura,
	 CONVERT(varchar,T1.OrderDate,105) AS Fecha,
	 T0.ProductID AS ProductID,
	 T2.Name AS Producto,
	 T0.OrderQty AS Cantidad,
	 T0.UnitPrice AS Precio
  FROM [Sales].[SalesOrderDetail] T0
  join sales.SalesOrderHeader T1 ON T0.SalesOrderID = T1.SalesOrderID
  join Production.Product T2 ON T0.ProductID = T2.ProductID
	   '''

data = pd.read_sql(input_query, conn_str)

### Exploración inicial de los datos

In [37]:
data.shape

(121317, 6)

In [38]:
data.columns

Index(['Factura', 'Fecha', 'ProductID', 'Producto', 'Cantidad', 'Precio'], dtype='object')

In [39]:
data.head(10)

,Factura,Fecha,ProductID,Producto,Cantidad,Precio
0,43659,31-05-2011,776,"Mountain-100 Black, 42",1,2024.9940
1,43659,31-05-2011,777,"Mountain-100 Black, 44",3,2024.9940
2,43659,31-05-2011,778,"Mountain-100 Black, 48",1,2024.9940
3,43659,31-05-2011,771,"Mountain-100 Silver, 38",1,2039.9940
4,43659,31-05-2011,772,"Mountain-100 Silver, 42",1,2039.9940
5,43659,31-05-2011,773,"Mountain-100 Silver, 44",2,2039.9940
6,43659,31-05-2011,774,"Mountain-100 Silver, 48",1,2039.9940
7,43659,31-05-2011,714,"Long-Sleeve Logo Jersey, M",3,28.8404
8,43659,31-05-2011,716,"Long-Sleeve Logo Jersey, XL",1,28.8404
9,43659,31-05-2011,709,"Mountain Bike Socks, M",6,5.7000


### Preprocesamiento de datos

In [40]:
# quitando espacios extras en el nombre del producto
data['Producto'] = data['Producto'].str.strip()

In [41]:
# verificando los tipos de variables
data.dtypes

Factura        int64
Fecha         object
ProductID      int64
Producto      object
Cantidad       int64
Precio       float64
dtype: object

In [42]:
# quitando filas sin numero de factura
data.dropna(axis = 0, subset =['Factura'], inplace = True)
data['Factura'] = data['Factura'].astype('str')

### Procesamiento de los datos

#### Creando el objeto de tipo transacciones

In [43]:
basket = (data
		.groupby(['Factura', 'Producto'])['Cantidad']
		.sum().unstack().reset_index().fillna(0)
		.set_index('Factura'))

#### Explorando el objeto basket

In [44]:
basket.head(10)

Producto,AWC Logo Cap,All-Purpose Bike Stand,Bike Wash - Dissolver,Cable Lock,Chain,"Classic Vest, L","Classic Vest, M","Classic Vest, S",Fender Set - Mountain,Front Brakes,...,"Touring-3000 Yellow, 54","Touring-3000 Yellow, 58","Touring-3000 Yellow, 62",Water Bottle - 30 oz.,"Women's Mountain Shorts, L","Women's Mountain Shorts, M","Women's Mountain Shorts, S","Women's Tights, L","Women's Tights, M","Women's Tights, S"
Factura,,,,,,,,,,,,,,,,,,,,,
43659,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
43660,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
43661,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
43662,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
43663,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
43664,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
43665,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
43666,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
43667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Definición de la función de codificación en caliente para que los datos sean adecuados con valores entre 0 y 1

In [45]:
def hot_encode(x):
	if(x<= 0):
		return 0
	if(x>= 1):
		return 1

#### Encoding el dataset

In [46]:
basket_encoded = basket.applymap(hot_encode)
basket = basket_encoded

Este código es utilizado para transformar una matriz de compras (basket) en una matriz binaria de hot encoding.

La función hot_encode transforma cada valor en la matriz en 1 si el valor existe en esa transacción o 0 si no existe. El método applymap aplica la función hot_encode a cada elemento de la matriz basket. Por lo tanto, basket_encoded es una matriz binaria de hot encoding.

Finalmente, la matriz original basket se actualiza para que sea igual a basket_encoded.

### Creación del modelo de Reglas de Asociación

In [47]:
frq_items = apriori(basket, min_support = 0.03, use_colnames = True)

Cuando se aplica el algoritmo Apriori para obtener reglas de asociación, se pueden obtener varias métricas que nos ayudan a evaluar la calidad y la relevancia de las reglas encontradas. Algunas de las métricas más comunes son:

* **Soporte**: Es la proporción de transacciones en las que aparece un conjunto de items, es decir, la frecuencia relativa con la que se presenta el conjunto de items en la base de datos. Se calcula dividiendo el número de transacciones que contienen el conjunto de items por el número total de transacciones. Un valor alto de soporte indica que el conjunto de items es frecuente en la base de datos.

* **Confianza**: Es la probabilidad condicional de que un item X aparezca en una transacción, dado que la transacción ya contiene otro item Y. Se calcula dividiendo el número de transacciones que contienen ambos items por el número de transacciones que contienen el item Y. Un valor alto de confianza indica que si el item Y aparece en una transacción, es probable que también aparezca el item X.

* **Lift**: Es una medida de la correlación entre dos items. Se calcula dividiendo la confianza de una regla por la frecuencia relativa del item que aparece en la conclusión de la regla. Un valor de lift mayor a 1 indica que la presencia del item Y en una transacción aumenta la probabilidad de que aparezca el item X, lo que indica una correlación positiva. Un valor de lift menor a 1 indica una correlación negativa, mientras que un valor igual a 1 indica que no hay correlación entre los items.

* **Convicción**: Es una medida de la dependencia entre dos items, que se calcula como la ratio entre la confianza de la regla y la complementaria de la frecuencia relativa del item en la conclusión de la regla. Un valor alto de convicción indica que si el item Y no aparece en una transacción, es poco probable que el item X aparezca, lo que indica una dependencia fuerte entre los items.

Estas métricas son útiles para evaluar y seleccionar las reglas de asociación más relevantes y útiles en un conjunto de datos. Por ejemplo, se pueden utilizar para identificar patrones de compra en un negocio, o para analizar patrones de uso en una aplicación web.

#### Explorando el modelo creado

In [48]:
frq_items.head(10)

,support,itemsets
0,0.107485,(AWC Logo Cap)
1,0.042174,(Bike Wash - Dissolver)
2,0.067408,(Fender Set - Mountain)
3,0.044367,(HL Mountain Tire)
4,0.034515,"(Half-Finger Gloves, M)"
5,0.034133,(Hydration Pack - 70 oz.)
6,0.033180,(LL Road Tire)
7,0.051962,"(Long-Sleeve Logo Jersey, L)"
8,0.038710,"(Long-Sleeve Logo Jersey, M)"
9,0.034197,"(Long-Sleeve Logo Jersey, XL)"


#### Recopilación de las reglas inferidas en un marco de datos

In [49]:
rules = association_rules(frq_items, metric ="lift", min_threshold = 0.5)
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False])

In [50]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
15,(Road Bottle Cage),(Water Bottle - 30 oz.),0.054410,0.148991,0.048339,0.888435,5.963011,0.040233,7.627893
13,(Mountain Bottle Cage),(Water Bottle - 30 oz.),0.064357,0.148991,0.053774,0.835556,5.608096,0.044185,5.175055
1,"(Long-Sleeve Logo Jersey, L)",(AWC Logo Cap),0.051962,0.107485,0.037248,0.716820,6.669050,0.031663,3.151756
10,(HL Mountain Tire),(Mountain Tire Tube),0.044367,0.098363,0.030415,0.685530,6.969371,0.026051,2.867164
12,(Water Bottle - 30 oz.),(Mountain Bottle Cage),0.148991,0.064357,0.053774,0.360922,5.608096,0.044185,1.464050
0,(AWC Logo Cap),"(Long-Sleeve Logo Jersey, L)",0.107485,0.051962,0.037248,0.346541,6.669050,0.031663,1.450798
3,"(Sport-100 Helmet, Black)",(AWC Logo Cap),0.095567,0.107485,0.031400,0.328567,3.056875,0.021128,1.329269
5,"(Sport-100 Helmet, Blue)",(AWC Logo Cap),0.098204,0.107485,0.032131,0.327184,3.044015,0.021575,1.326538
14,(Water Bottle - 30 oz.),(Road Bottle Cage),0.148991,0.054410,0.048339,0.324445,5.963011,0.040233,1.399725
7,"(Sport-100 Helmet, Red)",(AWC Logo Cap),0.097982,0.107485,0.030351,0.309763,2.881934,0.019820,1.293057


## Conclusiones

Después de realizar el análisis de asociación mediante el algoritmo *Apriori*, se pueden obtener conclusiones útiles para la empresa **AdventureWorks**.

* En primer lugar, se pueden identificar cuáles son los productos que se venden juntos con mayor frecuencia, lo que puede ayudar a la empresa a hacer recomendaciones cruzadas de productos y aumentar las ventas. Además, se pueden descubrir combinaciones de productos que no se venden con frecuencia, lo que puede ser una oportunidad para la empresa para promocionarlos y mejorar su venta.

* También se pueden identificar patrones en las ventas a lo largo del tiempo, lo que puede ayudar a la empresa a planificar su inventario y gestionar la cadena de suministro de manera más eficiente.

En resumen, el análisis de asociación puede ser una herramienta valiosa para las empresas que desean mejorar sus ventas y optimizar su inventario.

## Recomendaciones

A partir de los resultados obtenidos en el análisis, se pueden dar las siguientes recomendaciones a la empresa **AdventureWorks**:

* Promocionar los productos que tienen mayor soporte y confianza en los clientes, ya que son los que tienen mayor probabilidad de ser comprados en conjunto.

* Investigar la relación entre los productos que tienen menor soporte y confianza, y ver si se pueden tomar medidas para mejorar su atractivo para los clientes.

* Analizar las combinaciones de productos que no aparecieron en las reglas de asociación pero que podrían tener potencial para ser vendidos juntos, ya que podrían ser una oportunidad de generar ingresos adicionales.

* Segmentar a los clientes en función de sus patrones de compra y personalizar las ofertas de productos para cada segmento, lo que podría aumentar la satisfacción del cliente y la retención.

* Realizar análisis de asociación periódicamente para identificar cambios en los patrones de compra de los clientes y ajustar la estrategia de marketing en consecuencia.

* Considerar el uso de herramientas de recomendación personalizadas para promover productos adicionales que puedan interesar a los clientes en función de su historial de compras.

En general, la empresa podría aprovechar los datos de ventas históricos para mejorar su estrategia de marketing y promoción, y de esta manera aumentar sus ventas y la satisfacción del cliente.